In [3]:
#Check if container is passing through + general imports 
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [28]:
#Make Databricks connection
from databricks_connect import connect_explicit

spark = connect_explicit()

In [51]:
# Cleaning and joining data for demand forecasts
data = spark.sql("""
    SELECT 
        ss_sold_date_sk,
        ss_item_sk,
        ss_store_sk,
        ss_promo_sk,
        ss_quantity,
        d_date,
        d_holiday,
        d_following_holiday,
        d_weekend,
        i_item_id,
        s_store_id
        
    FROM samples.tpcds_sf1.store_sales AS ss
    INNER JOIN samples.tpcds_sf1.date_dim AS dd
    ON ss.ss_sold_date_sk = dd.d_date_sk
    INNER JOIN samples.tpcds_sf1.item AS i
    ON ss.ss_item_sk = i.i_item_sk
    INNER JOIN samples.tpcds_sf1.store AS s
    ON ss.ss_store_sk = s.s_store_sk
    """)

# Promotion table with actual dates
promo = spark.sql("""
    SELECT
        p.p_item_sk, 
        p.p_promo_sk,
        p.p_promo_id,
        dd_start.d_date as promo_start_date,
        dd_end.d_date as promo_end_date

    FROM samples.tpcds_sf1.promotion AS p
    LEFT JOIN samples.tpcds_sf1.date_dim AS dd_start
    ON p.p_start_date_sk = dd_start.d_date_sk
    LEFT JOIN samples.tpcds_sf1.date_dim AS dd_end
    ON p.p_end_date_sk = dd_end.d_date_sk
    """)

sales_promo = data.join(
    promo,
    (data.ss_item_sk == promo.p_item_sk) & 
    (data.d_date >= promo.promo_start_date) & 
    (data.d_date <= promo.promo_end_date),
    "left"
).select(
    data["*"],
    promo.p_promo_id,
    promo.promo_start_date,
    promo.promo_end_date
)

sales_promo.show()


+---------------+----------+-----------+-----------+-----------+----------+---------+-------------------+---------+----------------+----------------+----------+----------------+--------------+
|ss_sold_date_sk|ss_item_sk|ss_store_sk|ss_promo_sk|ss_quantity|    d_date|d_holiday|d_following_holiday|d_weekend|       i_item_id|      s_store_id|p_promo_id|promo_start_date|promo_end_date|
+---------------+----------+-----------+-----------+-----------+----------+---------+-------------------+---------+----------------+----------------+----------+----------------+--------------+
|        2451181|     14386|          1|        251|         77|1999-01-02|        N|                  Y|        Y|AAAAAAAACDIDAAAA|AAAAAAAABAAAAAAA|      NULL|            NULL|          NULL|
|        2451181|     11323|          1|          1|         84|1999-01-02|        N|                  Y|        Y|AAAAAAAALDMCAAAA|AAAAAAAABAAAAAAA|      NULL|            NULL|          NULL|
|        2451181|     10141|       

In [50]:
sales_promo = data.join(
    promo,
    (data.ss_item_sk == promo.p_item_sk) & 
    (data.d_date >= promo.promo_start_date) & 
    (data.d_date <= promo.promo_end_date),
    "left"
).select(
    data["*"],
    promo.p_promo_id,
    promo.promo_start_date,
    promo.promo_end_date
)

sales_promo.show()

+---------------+----------+-----------+-----------+-----------+----------+---------+-------------------+---------+----------------+----------------+----------+----------------+--------------+
|ss_sold_date_sk|ss_item_sk|ss_store_sk|ss_promo_sk|ss_quantity|    d_date|d_holiday|d_following_holiday|d_weekend|       i_item_id|      s_store_id|p_promo_id|promo_start_date|promo_end_date|
+---------------+----------+-----------+-----------+-----------+----------+---------+-------------------+---------+----------------+----------------+----------+----------------+--------------+
|        2451181|     14386|          1|        251|         77|1999-01-02|        N|                  Y|        Y|AAAAAAAACDIDAAAA|AAAAAAAABAAAAAAA|      NULL|            NULL|          NULL|
|        2451181|     11323|          1|          1|         84|1999-01-02|        N|                  Y|        Y|AAAAAAAALDMCAAAA|AAAAAAAABAAAAAAA|      NULL|            NULL|          NULL|
|        2451181|     10141|       